# Third model 

This model will try to predict wether pe is present or not on a full image (RGB level)

## Imports

In [1]:
import numpy as np
from balance import *
from model import *
from torchsummary import summary

In [2]:
import torch
torch.cuda.empty_cache()

print(f'PyTorch version: {torch.__version__}')
print("GPU found :)" if torch.cuda.is_available() else "No GPU :(")

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

PyTorch version: 1.7.1
GPU found :)


# Load datasets

In [3]:
full_image_dataset = torch.load('datasets/full_image_dataset.pt')

## CNN

In [4]:
class ModelV3(nn.Module):
    
    def __init__(self, input_channels):
        """Convnet with 4 convolution layer + pooling + BN, with 3 fully connected at the end"""
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 64, 3)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, 3)
        self.bn4 = nn.BatchNorm2d(64)
        
        self.pool = nn.MaxPool2d(kernel_size=2)
        
        self.fc1 = nn.Linear(64*14*14 , 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 1)
        
        self.dropout = nn.Dropout(0.5)
        
        self.sigmoid = nn.Sigmoid()
        
        
    def forward(self, x):
        
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = x.view(-1, 64*14*14) # Flatten
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        
        # Binary classification
        out = self.sigmoid(self.fc3(x))
        
        return out

In [5]:
model = ModelV3(3).to(device)
summary(model, (3, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 252, 252]           2,432
       BatchNorm2d-2         [-1, 32, 252, 252]              64
         MaxPool2d-3         [-1, 32, 126, 126]               0
            Conv2d-4         [-1, 64, 124, 124]          18,496
       BatchNorm2d-5         [-1, 64, 124, 124]             128
         MaxPool2d-6           [-1, 64, 62, 62]               0
            Conv2d-7           [-1, 64, 60, 60]          36,928
       BatchNorm2d-8           [-1, 64, 60, 60]             128
         MaxPool2d-9           [-1, 64, 30, 30]               0
           Conv2d-10           [-1, 64, 28, 28]          36,928
      BatchNorm2d-11           [-1, 64, 28, 28]             128
        MaxPool2d-12           [-1, 64, 14, 14]               0
          Dropout-13                [-1, 12544]               0
           Linear-14                  [

## Split data into train and test

In [6]:
batch_size = 8
ratio=0.2

train_loader, test_loader = train_test_split(full_image_dataset, ratio, batch_size)

## Model Training

In [7]:
num_epoch = 30
criterion = nn.BCELoss()
optimizer=optim.Adam(model.parameters(), lr=0.001)
save_filename = './models/model_v3.h5'

model, losses = train(model, num_epoch, criterion, train_loader, optimizer, device, save_filename)

Epoch [1/20], step [10/157], loss: 0.8548
Epoch [1/20], step [20/157], loss: 0.4556
Epoch [1/20], step [30/157], loss: 0.9078
Epoch [1/20], step [40/157], loss: 1.0192
Epoch [1/20], step [50/157], loss: 0.2572
Epoch [1/20], step [60/157], loss: 0.4021
Epoch [1/20], step [70/157], loss: 0.9743
Epoch [1/20], step [80/157], loss: 0.4781
Epoch [1/20], step [90/157], loss: 0.3997
Epoch [1/20], step [100/157], loss: 0.5169
Epoch [1/20], step [110/157], loss: 0.4380
Epoch [1/20], step [120/157], loss: 0.3004
Epoch [1/20], step [130/157], loss: 0.5137
Epoch [1/20], step [140/157], loss: 0.7301
Epoch [1/20], step [150/157], loss: 0.6277
Epoch [2/20], step [10/157], loss: 0.7538
Epoch [2/20], step [20/157], loss: 0.6407
Epoch [2/20], step [30/157], loss: 0.4411
Epoch [2/20], step [40/157], loss: 0.3688
Epoch [2/20], step [50/157], loss: 0.4121
Epoch [2/20], step [60/157], loss: 0.3587
Epoch [2/20], step [70/157], loss: 0.5705
Epoch [2/20], step [80/157], loss: 0.3666
Epoch [2/20], step [90/157],

In [ ]:
plot_loss(losses, num_epoch, len(train_loader), 'Loss model v3 full image.png')

## Test the model

In [9]:
# Load model for evaluation
trained_model = ModelV3(input_channels=3)
trained_model.load_state_dict(torch.load('./models/model_v3.h5'))
trained_model.to(device)

ModelV3(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (bn4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=12544, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (sigmoid): Sigmoid()
)

In [10]:
accuracy, total = evaluate(trained_model, test_loader, device)

In [11]:
print(f'Accuracy of the network on the {total} test images: {accuracy:.2f}')

Accuracy of the network on the 1254 test images: 83.89
